## FFE geo-join - Geomesa performance test

This geo-join operation creates a set of edges for buildings where the distance between centers < N. 

Although the final solution requires more accurate distance between buildings this first pass will drastically reduce the number of objects that ultimately need to be handled with a more detailed approach.

This is a very costly O(n2) operation withoout some geospatial indexing. This is where Spark + Geomesa can help.

Spark prvides the parallisation framework, allowing all the available CPU to be used.

Geomesa provides Spark-compatible geo-function that use geo-indexing to greatly reduce compute costs. 


In [2]:
from datetime import datetime as dt
import geomesa_pyspark

from pyspark.sql import Row
conf = geomesa_pyspark.configure().setAppName('Demo1')

from pyspark.sql import SparkSession

spark = SparkSession.builder.enableHiveSupport().getOrCreate()
sc = spark.sparkContext

# geomesa_pyspark.init_sql(spark) in later version 2.4 DOCS)
spark._jvm.org.apache.spark.sql.SQLTypes.init(spark._jwrapped)

In [3]:
filename = "file:///home/jovyan/DEMO/buildings_raw.csv"

df = spark.read \
    .option("header", True) \
    .option("inferSchema", True) \
    .csv(filename)

In [4]:
newdf = df.repartition(72)
newdf.createOrReplaceTempView('df')

In [5]:
%%time
def timed_join(sample_size=1000):
    t0 = dt.utcnow()
    qry = "SELECT df1._c0 as id, " +\
        "df2._c0 as near_id " +\
        "FROM df as df1, df as df2 " +\
        "WHERE st_distance(st_point(df1.X, df1.Y), st_point(df2.X, df2.Y)) < 50 "+\
        "AND df1._c0 <> df2._c0 " +\
        "AND df1._c0 < %s and df2._c0 < %s" % (sample_size, sample_size)
    result = spark.sql(qry)
    count = result.count()
    print("join + count %s took %s with %s edges" % (sample_size, dt.utcnow()-t0, count))
    # print('ratio', result.count()/(df.count() **2))
    return result

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 16.5 µs


In [6]:
for n in [1e3, 2e3, 4e3]: #, 8e3, 1e4, 2e4, 4e4, 8e4]:
    edges = timed_join(int(n))
edges.cache()
edges.show()

join + count 1000 took 0:00:11.302566 with 3966 edges
join + count 2000 took 0:00:02.761756 with 14148 edges
join + count 4000 took 0:00:06.073014 with 37052 edges
+----+-------+
|  id|near_id|
+----+-------+
|2909|   2905|
|2909|   2899|
|2909|   2900|
|2909|   2895|
|2909|   2902|
|2909|   2903|
|2909|   2898|
|2909|   2872|
|2909|   2904|
|2909|   2901|
|2355|   1977|
|2355|   2346|
|2355|   2347|
|2041|   2051|
|2041|   2396|
|2041|   2018|
|2041|   2042|
|2041|   2021|
|2041|   2020|
|2041|   2045|
+----+-------+
only showing top 20 rows



In [ ]:
# %%time
# qry = "SELECT df1._c0 as id, df1.suburb_loc as suburb, df1.Combustibl as combustible, " +\
#     "df2._c0 as near_id, st_distance(st_point(df1.X, df1.Y), st_point(df2.X, df2.Y)) as distance " +\
#     "FROM df as df1, df as df2 " +\
#     "WHERE st_distance(st_point(df1.X, df1.Y), st_point(df2.X, df2.Y)) < 50"
# result = spark.sql(qry)
# # print('ratio', result.count()/(df.count() **2))
# # result.show()

In [ ]:
#from https://github.com/locationtech/rasterframes/blob/43bd3b37f0b2931470b2d5b5757474d3c885e659/pyrasterframes/src/main/python/pyrasterframes/rasterfunctions.py#L718-L1062

# from __future__ import absolute_import
# from pyspark.sql.column import Column, _to_java_column
# from pyspark.sql.functions import lit
# # from .rf_context import RFContext
# # from .rf_types import CellType, Extent, CRS

# def _apply_column_function(name, *args):
# #     jfcn = RFContext.active().lookup(name)
#     jcols = [_to_java_column(arg) for arg in args]
#     return Column(*jcols)

# def st_polygonFromText(*args):
#     """"""
#     return _apply_column_function('st_polygonFromText', *args)


In [ ]:
# from pyrasterframes.utils import create_rf_spark_session
# from pyrasterframes.rasterfunctions import *
# # spark = create_rf_spark_session()
# # st_polygonFromText
# import pyspark.sql.functions as fn

In [ ]:
# import geomesa_pyspark as gpk

In [ ]:
# help(gpk.spark.GeoMesaSpark.apply)

In [ ]:
class MyContext(object):
    """
    Entrypoint to RasterFrames services
    """
    def __init__(self, spark_session):
        self._spark_session = spark_session
        self._gateway = spark_session.sparkContext._gateway
        self._jvm = self._gateway.jvm
        #jsess = self._spark_session._jsparkSession
        #self._jrfctx = self._jvm.org.locationtech.rasterframes.py.PyRFContext(jsess)
        self.context = self._jvm.org.locationtech.geomesa.spark.jts.DataFrameFunctions
        
    def lookup(self, function_name: str):
        return getattr(self._jrfctx, function_name)        
